In [1]:
import yfinance as yf
import polars as pl
from typing import Union, List

def load_yahoo_data(tickers: Union[str, List[str]]) -> pl.DataFrame:
    """
    Load closing price data for one or multiple tickers from Yahoo Finance.
    Returns the maximum available data for all tickers.
    
    Args:
    tickers (str or list of str): Single ticker or list of tickers to fetch data for.
    
    Returns:
    polars.DataFrame: DataFrame containing the closing prices for all tickers.
    """
    if isinstance(tickers, str):
        tickers = [tickers]
    
    data_frames = []
    for ticker in tickers:
        df = yf.Ticker(ticker).history(period="max")
        df = pl.from_pandas(df.reset_index())
        df = df.select([pl.col("Date"), pl.col("Close").alias(ticker)])
        data_frames.append(df)
    
    merged_df = data_frames[0]
    for df in data_frames[1:]:
        merged_df = merged_df.join(df, on="Date", how="inner")
    
    return merged_df.sort("Date")

# For a single ticker
df_single = load_yahoo_data("AAPL")

# For multiple tickers
df_multiple = load_yahoo_data(["AAPL", "GOOGL", "MSFT"])

In [3]:
df_single

Date,AAPL
"datetime[ns, America/New_York]",f64
1980-12-12 00:00:00 EST,0.099058
1980-12-15 00:00:00 EST,0.09389
1980-12-16 00:00:00 EST,0.086999
1980-12-17 00:00:00 EST,0.089152
1980-12-18 00:00:00 EST,0.091737
…,…
2024-06-28 00:00:00 EDT,210.619995
2024-07-01 00:00:00 EDT,216.75
2024-07-02 00:00:00 EDT,220.270004
